## Q3 (5 pts)

Please fill out the following project form (one per group). This will primarily be for the TAs to use to understand what you are working on and hopefully be able to better assist you. If you haven't decided on certain aspects of the project, just include what you are currently thinking/what decisions you need to make.


**(1) Write down your dynamics (handwritten, code, or latex). This can be continuous-time (include how you are discretizing your system) or discrete-time.**

$$
\begin{align}
    \begin{bmatrix} \ddot x \\ \ddot y \\ \ddot z \end{bmatrix}
    &= {}_V^O R(\alpha, \beta, \gamma) \begin{bmatrix} 0 \\ 0 \\ a \end{bmatrix} + \begin{bmatrix} 0 \\ 0 \\ -g \end{bmatrix} \\
    \begin{bmatrix} \dot \gamma \\ \dot \beta \\ \dot \alpha \end{bmatrix}
    &= \begin{bmatrix} \cos\beta \cos\gamma & -\sin\gamma & 0 \\ \cos\beta \sin\gamma & \cos\gamma & 0 \\ -\sin\beta & 0 & 1 \end{bmatrix}^{-1} \begin{bmatrix} \omega_x \\ \omega_y \\ \omega_z \end{bmatrix} \\
    % \begin{bmatrix} \ddot r \\ \ddot s\end{bmatrix} &= f(r,s,\dot r, \dot s, \ddot x, \ddot y, \ddot z) \\
    \ddot r &= \frac{1}{(L^2-s^2)\zeta^2} \bigg( -r^4 \ddot x - (L^2 - s^2)^2 \ddot x - 2r^2(s\dot r\dot s + (-L^2 + s^2)\ddot x) + r^3 (\dot s^2 + s\ddot s \\
    &\;\;\;\; - \zeta(g + \ddot z)) + r(-L^2 s\ddot s + s^3 \ddot s + s^2(\dot r^2 - \zeta (g + \ddot z)) + L^2 (-\dot r^2 - \dot s^2 + \zeta(g+\ddot z)))\bigg) \\
    \ddot s &= \frac{1}{(L^2-r^2)\zeta^2} \bigg( -s^4 \ddot y - (L^2 - r^2)^2 \ddot y - 2s^2(r\dot r\dot s + (-L^2 + r^2)\ddot y) + s^3 (\dot r^2 + r\ddot r \\
    &\;\;\;\; - \zeta(g + \ddot z)) + s(-L^2 r\ddot r + r^3 \ddot r + r^2(\dot s^2 - \zeta (g + \ddot z)) + L^2 (-\dot r^2 - \dot s^2 + \zeta(g+\ddot z)))\bigg)
\end{align}
$$
where ${}_V^O R(\alpha, \beta, \gamma) = R_z(\alpha) R_y(\beta) R_x(\gamma)$ is the rotation matrix from the vehicle coordinate system $V$ to the inertial coordinate system $O$.

The continuous-time system is discretized using RK4 for low-fidelity simulation purposes. For control purposes, we are using the exponential method to discretize the system. For high-fidelity PX4 SITL, Gazebo is used.

**(2) What is your state (what does each variable represent)?**

State: 
$$
    \begin{bmatrix} 
        x \\ y \\ z \\ \dot x \\ \dot y \\ \dot z \\ \alpha \\ \beta \\ \gamma \\ r \\ s \\ \dot r \\ \dot s
    \end{bmatrix} = \begin{bmatrix}
        \text{Translational x-position in the inertial coordinate system }O. \\
        \text{Translational y-position in the inertial coordinate system }O. \\
        \text{Translational z-position in the inertial coordinate system }O. \\
        \text{Translational x-velocity in the inertial coordinate system }O. \\
        \text{Translational y-velocity in the inertial coordinate system }O. \\
        \text{Translational z-velocity in the inertial coordinate system }O. \\
        \text{Vehicle yaw angle.} \\
        \text{Vehicle pitch angle.} \\
        \text{Vehicle roll angle.} \\
        \text{Translational x-position of the pendulum CoM relative to its base in }O.\\
        \text{Translational y-position of the pendulum CoM relative to its base in }O.\\
        \text{Translational x-velocity of the pendulum CoM relative to its base in }O.\\
        \text{Translational y-velocity of the pendulum CoM relative to its base in }O.\\
    \end{bmatrix}
$$

We define $L$ as the length from the base of the pendulum to its center of mass (CoM). Furthermore, the relative position of the pendulum along the $z$-axis is $\zeta := \sqrt{L^2 - r^2 - s^2}$.

**(3) What is your control (what does each variable represent)?**

Control:
$$
    \begin{bmatrix}
        \omega_x \\
        \omega_y \\
        \omega_z \\
        a
    \end{bmatrix} = \begin{bmatrix}
        \text{Rotational rates about the vehicle body x-axis.} \\
        \text{Rotational rates about the vehicle body y-axis.} \\
        \text{Rotational rates about the vehicle body z-axis.} \\
        \text{Mass-normalized collective thrust in the vehicle body frame.}
    \end{bmatrix}
$$

**(4) Briefly describe your goal for the project. What are you trying to make the system do? Specify whether you are doing control, trajectory optimization, both, or something else.**

We are doing controls to balance an inverted pendulum on a quadrotor. The eventual goal is to use LQR to achieve this objective on a real hardware system. Making a constant position tracking LQR work on a real quadrotor is the objective.

If we have time, we would like to apply a neural control barrier function on this hardware system.

**(5) What are your costs?**

We are using infinite-horizon LQR for our controller.

$$ J = \int_0^\infty x(t)^T Q x(t) + u(t)^T R u(t) dt$$

**(6) What are your constraints?**

We have the usual dynamics constraint, initial condition constraint (not explicitly imposed, but for testing purposes), and control limits (not considered in LQR).

For the purpose of balancing this inverted pendulum, the constraint is that the pendulum cannot tip more than $\pi/6$ radians in roll or pitch. I.e., $r^2 \le (L\cos(\pi/6))^2$ and $s^2 \le (L\cos(\pi/6))^2$.

If we have time, i.e., our system works well in hardware with LQR, then we will proceed to the neural control barrier function part of this project.
For the neural control barrier function, we are considering two different approaches. The first is defining a single safety specification that combines these two constraints together and synthesizing a limit-aware neural control barrier function. The second is defining multiple safety specifications and synthesizing a smooth-underapproximation of the combined control barrier function using RL. 

**(7) What solution methods are you going to try?**


For constant position tracking (balancing the inverted pendulum while maintaining a constant quadrotor position), we are using infinite-horizon discretized LQR controller. The gains are tuned in high-fidelity PX4 SITL in Gazebo and refined in hardware experiments. This is from "M. Hehn, R. D'Andrea, 'A flying inverted pendulum', 2011 (https://ieeexplore.ieee.org/document/5980244)".

For synthesizing a neural control barrier function, we will use one of two methods:

(1) "S. Liu, C. Liu, J. M. Dolan, 'Safe Control Under Input Limits with Neural Control Barrier Functions', 2022 (https://arxiv.org/abs/2211.11056)". 

(2) Make a novel method based on "T. G. Molnar, A. D. Ames, 'Composing Control Barrier Functions for Complex Safety Specifications', 2023 (https://ieeexplore.ieee.org/document/10345664)".

**(8) What have you tried so far?**

1. Built a custom PX4-based quadrotor. This was done using scrap material from multiple labs.
2. Made a custom CAD model of this quadrotor and completed PX4 SITL in Gazebo.
3. Offboard control using MAVROS.
4. Position estimation using Vicon mocap and onboard EKF.
5. Implementation of constant-position tracking LQR in low-fidelity Julia simulator.
6. Implementation of constant-position tracking LQR in high-fidelity PX4 SITL Gazebo simulator.
7. Tuning LQR in Gazebo.
8. Currently about to try this in hardware.

**(9) If applicable, what are you currently running into issues with?**

1. Control frequency. Due to the weight limits of the quadrotor, we are not using an onboard companion computer, but rather a wifi chip (ESP8266) for communication with ground station. This wifi chip cannot achieve our desired control frequency, which is 50 Hz.
2. LQR cost tuning in real life.
3. Pendulum position tracking. Mocap beads positioning on the pendulum is difficult in real life.
4. Thrust control needs much tuning.
5. Formulation and training of neural control barrier functions.
    - Training using Simin Liu's method in PyTorch.
    - How to formulate a new training method based on RL?

**(10) If your system doesn't fit with some of the questions above or there are additional things you'd like to elaborate on, please explain/do that here.**

N/A